In [1]:
import sys,os
sys.path.append(os.path.abspath(os.path.join('..', 'src')))

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from utils import *
from pandas.api.types import CategoricalDtype

%matplotlib inline

# StackOverflow 2020 Exploratory Data Analysis

In [10]:
df_schema = pd.read_csv("../data/processed/so_schema.csv")
df_schema.head()

,field,question_text
0,respondent,Randomized respondent ID number (not in order ...
1,main_branch,Which of the following options best describes ...
2,hobbyist,Do you code as a hobby?
3,age,What is your age (in years)? If you prefer not...
4,age1st_code,At what age did you write your first line of c...


In [2]:
df = pd.read_csv("../data/processed/so_data.csv")
df.head()

,respondent,main_branch,hobbyist,age,age1st_code,comp_freq,comp_total,converted_comp,country,currency_desc,...,survey_ease,survey_length,trans,undergrad_major,webframe_desire_next_year,webframe_worked_with,welcome_change,work_week_hrs,years_code,years_code_pro
0,1,I am a developer by profession,Yes,NaN,13,Monthly,NaN,NaN,Germany,European Euro,...,Neither easy nor difficult,Appropriate in length,No,"Computer science, computer engineering, or sof...",ASP.NET Core,ASP.NET;ASP.NET Core,Just as welcome now as I felt last year,50.0,36,27
1,2,I am a developer by profession,No,NaN,19,NaN,NaN,NaN,United Kingdom,Pound sterling,...,NaN,NaN,NaN,"Computer science, computer engineering, or sof...",NaN,NaN,Somewhat more welcome now than last year,NaN,7,4
2,3,I code primarily as a hobby,Yes,NaN,15,NaN,NaN,NaN,Russian Federation,NaN,...,Neither easy nor difficult,Appropriate in length,NaN,NaN,NaN,NaN,Somewhat more welcome now than last year,NaN,4,NaN
3,4,I am a developer by profession,Yes,25.0,18,NaN,NaN,NaN,Albania,Albanian lek,...,NaN,NaN,No,"Computer science, computer engineering, or sof...",NaN,NaN,Somewhat less welcome now than last year,40.0,7,4
4,5,"I used to be a developer by profession, but no...",Yes,31.0,16,NaN,NaN,NaN,United States,NaN,...,Easy,Too short,No,"Computer science, computer engineering, or sof...",Django;Ruby on Rails,Ruby on Rails,Just as welcome now as I felt last year,NaN,15,8


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 64461 entries, 0 to 64460
Data columns (total 61 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   respondent                         64461 non-null  int64  
 1   main_branch                        64162 non-null  object 
 2   hobbyist                           64416 non-null  object 
 3   age                                45446 non-null  float64
 4   age1st_code                        57900 non-null  object 
 5   comp_freq                          40069 non-null  object 
 6   comp_total                         34826 non-null  float64
 7   converted_comp                     34756 non-null  float64
 8   country                            64072 non-null  object 
 9   currency_desc                      45472 non-null  object 
 10  currency_symbol                    45472 non-null  object 
 11  database_desire_next_year          44070 non-null  obj

## Examine Job Satisfaction

In [5]:
sat_type = CategoricalDtype(categories=["Very satisfied", "Slightly satisfied", "Neither satisfied nor dissatisfied", "Slightly dissatisfied", "Very dissatisfied"], ordered=True)
df.job_sat = df.job_sat.astype(sat_type)

df.job_sat.dtype

CategoricalDtype(categories=['Very satisfied', 'Slightly satisfied',
                  'Neither satisfied nor dissatisfied',
                  'Slightly dissatisfied', 'Very dissatisfied'],
, ordered=True)

In [6]:
df.job_sat.value_counts()

Very satisfied                        14611
Slightly satisfied                    13930
Slightly dissatisfied                  7124
Neither satisfied nor dissatisfied     5783
Very dissatisfied                      3746
Name: job_sat, dtype: int64

### Explore Null Job Satisfaction

In [13]:
df[df.job_sat.isnull()][["job_sat","dev_type", "job_factors", "job_seek", "employment"]]

,job_sat,dev_type,job_factors,job_seek,employment
2,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,Employed full-time
14,NaN,NaN,"Languages, frameworks, and other technologies ...","I’m not actively looking, but I am open to new...",Student
20,NaN,NaN,"Languages, frameworks, and other technologies ...","I’m not actively looking, but I am open to new...",Student
26,NaN,"Database administrator;Developer, full-stack;E...",NaN,NaN,Employed full-time
...,...,...,...,...,...
64456,NaN,Senior executive/VP,NaN,NaN,Employed full-time
64457,NaN,NaN,NaN,NaN,Employed full-time
64458,NaN,NaN,NaN,NaN,NaN
64459,NaN,NaN,NaN,NaN,Employed full-time


In [14]:
df.employment.value_counts()

Employed full-time                                      45270
Student                                                  7787
Independent contractor, freelancer, or self-employed     5672
Not employed, but looking for work                       2343
Employed part-time                                       2217
Not employed, and not looking for work                    322
Retired                                                   243
Name: employment, dtype: int64

**Explore Null Job Satisfaction for Currently Not Employed**

In [21]:
df[df.job_sat.notnull() & df.employment.isin(["Not employed, and not looking for work", "Student", "Not employed, but looking for work", "Retired"])]

,respondent,main_branch,hobbyist,age,age1st_code,comp_freq,comp_total,converted_comp,country,currency_desc,...,survey_ease,survey_length,trans,undergrad_major,webframe_desire_next_year,webframe_worked_with,welcome_change,work_week_hrs,years_code,years_code_pro
